General Health Query Chatbot (Prompt Engineering Based)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Mistral-7B-Instruct
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

print("Loading model... this may take a couple of minutes.")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
print("Model loaded successfully!")

Loading model... this may take a couple of minutes.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded successfully!


In [2]:
#Writting the instruction text (prompt) that tells the language model how to behave and how to answer.
def generate_prompt(user_query):
    return f"""
You are a helpful and friendly medical assistant.
You provide general health information for educational purposes only.
You must NOT recommend medicines, treatments, or home remedies by name.
You must NOT suggest taking any drugs.
If asked about treatment, explain general care principles and advise consulting a healthcare professional.

User question: "{user_query}"

Answer:
"""

In [3]:
#Talking to the language model and geting a response.
def get_health_response(user_query): #Calling previous function to get the full instruction text
    prompt = generate_prompt(user_query) #Converting text into numbers (tokens)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda") #Moving data to GPU and running inference faster using GPU
    output = model.generate(  #The model predicts the next words
        **inputs,
        max_new_tokens=200, #limiting tokens to prevent long output
        pad_token_id=tokenizer.eos_token_id
    )
    full_text = tokenizer.decode(output[0], skip_special_tokens=True) #Converting tokens back into readable English
    answer = full_text.replace(prompt, "").strip() #Avoiding system instructions from appearing in the response
    return answer


In [4]:
#This function protects users from unsafe medical advice, Checks if the response contains drug names or medical advice then uses keyword
#matching and overrides unsafe content with a safe message
def safety_filter(answer, user_query):
    risky_phrases = [ #these phrases will be onsidered as medical advice, which will be replaced by safety filter
        "how much should i take",
        "recommended dosage",
        "exact dose",
        "medicine amount",
        "treatment is",
        "cure for"
    ]
    if any(phrase in user_query.lower() for phrase in risky_phrases):
        return ( #If detected any above phrases will be replaced with following sentences
            "I can share general health information, but I can’t recommend "
            "for specific treatments or medications. It’s best to consult a qualified doctor."
        )
    return answer

In [5]:
print("Welcome to the General Health Chatbot! Type 'exit' to quit.")

while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Chatbot: Stay healthy! Goodbye.")
        break

    answer = get_health_response(user_query)
    safe_answer = safety_filter(answer, user_query)

    print(f"Chatbot: {safe_answer}\n")

Welcome to the General Health Chatbot! Type 'exit' to quit.
You: What causes a sore throat?
Chatbot: A sore throat can be caused by a variety of factors, including viral infections such as the common cold or flu, bacterial infections like strep throat, allergies, dry air, smoking, and acid reflux. It's important to note that I'm not a doctor and can't diagnose your specific condition. If you're experiencing a sore throat, it's best to consult a healthcare professional for an accurate diagnosis and appropriate treatment. In the meantime, staying hydrated, gargling with warm salt water, and using a humidifier may help alleviate symptoms.

You: Is paracetamol safe for children?
Chatbot: Paracetamol, also known as acetaminophen, is commonly used to relieve pain and reduce fever in children. However, it's important to use it carefully and follow the recommended dosage on the package or as directed by a healthcare professional. Overdosing can lead to serious health problems. Always consult a

Explanation of Results and Final Insights

The General Health Query Chatbot successfully demonstrates how a large language model (Mistral-7B-Instruct) can be guided to provide educational, safe health information using prompt engineering.

The safety filter effectively prevented the model from giving unsafe advice or recommending drugs, ensuring the chatbot remain informative but ethically safe.

Key insights:
Prompt engineering is crucial: Carefully designed prompts control the model’s behavior and tone.
LLMs can provide educational health guidance without direct training on a medical dataset, as long as safety constraints are applied.
Post-processing safety filters are important for handling edge cases where the model may still attempt to provide treatment information.
The chatbot demonstrates a balance between informativeness and ethical responsibility, making it suitable for educational or informational purposes.